<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/C_adapter_seq_bn_P_3_C_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary
This takes an adapter type and creates a pretraining adapter, a classification adapter with no pretrainng and a classification adapter from the pretraining adapter





# Setup

In [1]:
from time import time
start_time = time()

In [2]:
# Pretraining Dataset Settings
pretrain_dataset_name = "BigTMiami/amazon_split_25M_reviews_20_percent_condensed"
# 5% of 20% creates a 1% - I think this should take 15 minutes
pretrain_percentage_int = 15
pretrain_percentage = f"{pretrain_percentage_int}"
pretrain_train_split = f"train[:{pretrain_percentage}%]"
pretrain_eval_split = "validation"
# The dataset I am using is only 20%, so the final is 20% of the pretrain_percentage_int
pretrain_percentage_final = int(0.2 * pretrain_percentage_int)

# Pretraining Adapter settings
experiment_tag = "C"
adapter_type = "seq_bn" # could be "lora", etc.
# This uses SeqBnConfig() config - I don't change any of the parameters

pretrained_adapter_hub_name = f"{experiment_tag}_adapter_{adapter_type}_pretraining_P_{pretrain_percentage}"
pretrained_adapter_name = f"{pretrained_adapter_hub_name}_adapter"
print(f"Pretrained Adapter Hub Name: {pretrained_adapter_hub_name}")

# Classification Dataset Settings
classification_dataset_name = "BigTMiami/amazon_helpfulness"
classification_train_percentage = "20"
classification_train_split = f"train[:{classification_train_percentage}%]"
classification_eval_split = "dev"

classification_adapter_hub_name = f"{experiment_tag}_adapter_{adapter_type}_classification_C_{classification_train_percentage}"
classification_adapter_name = f"{classification_adapter_hub_name}_adapter"
print(f"Classifer Not Pretrained Adapter Hub Name: {classification_adapter_hub_name}")

# Classification using Pretrained Settings
classification_adapter_from_pretrained_hub_name = f"{experiment_tag}_adapter_{adapter_type}_classification_P_{pretrain_percentage}_to_C_{classification_train_percentage}"
classification_adapter_from_pretrained_name = f"{classification_adapter_from_pretrained_hub_name}_adapter"
print(f"Classifer from Pretrained Adapter Hub Name: {classification_adapter_from_pretrained_hub_name}")


Pretrained Adapter Hub Name: C_adapter_seq_bn_pretraining_P_15
Classifer Not Pretrained Adapter Hub Name: C_adapter_seq_bn_classification_C_20
Classifer from Pretrained Adapter Hub Name: C_adapter_seq_bn_classification_P_15_to_C_20


In [3]:
# from google.colab import drive
# drive.mount("/content/drive")
# %cd '/content/drive/MyDrive/AdaptOrDie'

In [4]:
!pip install datasets
!pip install transformers[torch]
!pip install adapters
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 7.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86

In [5]:
!pip install pynvml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.3 MB/s eta 0:00:00


In [6]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    gpu_used = info.used//1024**2
    print(f"GPU {gpu_used} MB")

print_gpu_utilization()

GPU 448 MB


In [7]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    f1 = f1_score(labels, preds, average='macro')

    return {
        'accuracy': accuracy,
        'f1_macro': f1
    }

# Pretraining

In [8]:
from datasets import load_dataset

pretrain_train_dataset, pretrain_eval_dataset = load_dataset(pretrain_dataset_name, split=[pretrain_train_split,pretrain_eval_split])

print(pretrain_train_dataset)
print(pretrain_eval_dataset)

Generating train split:   0%|          | 0/862683 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8360 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 129402
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 8360
})


In [9]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
pretrain_model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'heads.default.3.bias', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
from adapters import SeqBnConfig

# Not customizing this, using defaults
# adapter_config is used again for the classification no pretraining adapter
adapter_config = SeqBnConfig()

# Add a new adapter
pretrain_model.add_adapter(pretrained_adapter_name, config=adapter_config)

# Add head for masked language modeling
pretrain_model.add_masked_lm_head(pretrained_adapter_name)

# Set the adapter to be used for training
pretrain_model.train_adapter(pretrained_adapter_name)

print(f"Pretrained Adapter Name: {pretrained_adapter_name}")

Pretrained Adapter Name: C_adapter_seq_bn_pretraining_P_15_adapter


In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
print_gpu_utilization()

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

GPU 448 MB


In [12]:
from transformers import DataCollatorForLanguageModeling

pretraining_data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

In [13]:
from transformers import TrainingArguments

pretraining_args = TrainingArguments(
    output_dir=pretrained_adapter_hub_name,
    learning_rate=1e-4,
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=11,
    weight_decay=0.01,
    logging_steps=200,
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

In [14]:
from adapters import AdapterTrainer

pretrain_trainer = AdapterTrainer(
    model=pretrain_model,
    args=pretraining_args,
    train_dataset=pretrain_train_dataset,
    eval_dataset=pretrain_eval_dataset,
    data_collator=pretraining_data_collator,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [15]:
before_pretrain_eval_results = pretrain_trainer.evaluate()
before_pretrain_eval_results_loss = before_pretrain_eval_results["eval_loss"]
print(f"Before pretraining loss: {before_pretrain_eval_results_loss:.4f}")
print(before_pretrain_eval_results )

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Before pretraining loss: 16.7872
{'eval_loss': 16.787200927734375, 'eval_runtime': 91.8156, 'eval_samples_per_second': 91.052, 'eval_steps_per_second': 2.854}


In [16]:
pretrain_results = pretrain_trainer.train()
print(pretrain_results)

Step,Training Loss
200,5.032800


TrainOutput(global_step=367, training_loss=3.9401252601062247, metrics={'train_runtime': 2843.1815, 'train_samples_per_second': 45.513, 'train_steps_per_second': 0.129, 'total_flos': 3.485399605464269e+16, 'train_loss': 3.9401252601062247, 'epoch': 1.0})


In [17]:
after_pretrain_eval_results = pretrain_trainer.evaluate()
after_pretrain_eval_results_loss = after_pretrain_eval_results["eval_loss"]
print(f"After pretraining loss: {after_pretrain_eval_results_loss:.4f}")
print(after_pretrain_eval_results)

After pretraining loss: 2.4084
{'eval_loss': 2.408372640609741, 'eval_runtime': 90.8512, 'eval_samples_per_second': 92.019, 'eval_steps_per_second': 2.884, 'epoch': 1.0}


In [18]:
pretrain_model.push_adapter_to_hub(
    pretrained_adapter_hub_name,
    pretrained_adapter_name,
    datasets_tag=pretrain_dataset_name
)

pytorch_model_head.bin:   0%|          | 0.00/157M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_adapter.bin:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/BigTMiami/C_adapter_seq_bn_pretraining_P_15/commit/fdfdc9a9cc7bc90f949e5a9bf81df9275ff6b18a', commit_message='Upload model', commit_description='', oid='fdfdc9a9cc7bc90f949e5a9bf81df9275ff6b18a', pr_url=None, pr_revision=None, pr_num=None)

In [19]:
import gc

print_gpu_utilization()

del pretraining_data_collator
del pretrain_train_dataset
del pretrain_eval_dataset
del pretrain_model
del pretrain_trainer
gc.collect()

print_gpu_utilization()

GPU 27307 MB
GPU 27307 MB


# Classification

In [20]:
from datasets import load_dataset

classification_train_dataset, classification_eval_dataset= load_dataset(classification_dataset_name, split=[classification_train_split,classification_eval_split])

print(classification_train_dataset)
print(classification_eval_dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating dev split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 23050
})
Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 5000
})


In [21]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
classficiation_no_pretrain_model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'heads.default.3.bias', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
# adapter_config is created and set above with the pretraining adapter

# Add a new adapter
classficiation_no_pretrain_model.add_adapter(classification_adapter_name, config=adapter_config)

# Add head for classification modeling
classficiation_no_pretrain_model.add_classification_head(
    classification_adapter_name,
    num_labels=2,
    id2label={ 0: "unhelpful", 1: "helpful"})

# Set the adapter to be used for training
classficiation_no_pretrain_model.train_adapter(classification_adapter_name)

print(f"Classification Adapter Name: {classification_adapter_name}")

Classification Adapter Name: C_adapter_seq_bn_classification_C_20_adapter


In [23]:
from transformers import DataCollatorWithPadding

classification_data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [24]:
from transformers import TrainingArguments

classification_training_args = TrainingArguments(
    output_dir=classification_adapter_hub_name,
    learning_rate=1e-4,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=250,
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

In [25]:
from adapters import AdapterTrainer

classification_trainer = AdapterTrainer(
    model=classficiation_no_pretrain_model,
    args=classification_training_args,
    train_dataset=classification_train_dataset,
    eval_dataset=classification_eval_dataset,
    data_collator=classification_data_collator,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [26]:
classification_training_results = classification_trainer.train()
print(classification_training_results)

Step,Training Loss
250,0.407300
500,0.362500
750,0.368300
1000,0.335000
1250,0.324200
1500,0.354100
1750,0.329200
2000,0.334100
2250,0.329100
2500,0.327800


TrainOutput(global_step=4323, training_loss=0.33347256493022426, metrics={'train_runtime': 1108.842, 'train_samples_per_second': 62.362, 'train_steps_per_second': 3.899, 'total_flos': 1.7389360436066208e+16, 'train_loss': 0.33347256493022426, 'epoch': 3.0})


In [27]:
classification_eval_results = classification_trainer.evaluate()
classification_eval_loss = classification_eval_results["eval_loss"]
classification_eval_f1 = classification_eval_results["eval_f1_macro"]
print(f"Classification no Pretraining Eval F1:{100.0 * classification_eval_f1:.2f}")
print(f"Classification no Pretraining Eval Loss: {classification_eval_loss:.2f}")
print(classification_eval_results)

Classification no Pretraining Eval F1:62.37
Classification no Pretraining Eval Loss: 0.33
{'eval_loss': 0.3313445448875427, 'eval_accuracy': 0.8678, 'eval_f1_macro': 0.6237164946863191, 'eval_runtime': 37.6873, 'eval_samples_per_second': 132.671, 'eval_steps_per_second': 8.305, 'epoch': 3.0}


In [28]:
classficiation_no_pretrain_model.push_adapter_to_hub(
    classification_adapter_hub_name,
    classification_adapter_name,
    datasets_tag=classification_dataset_name
)
print(f"Pushed {classification_adapter_hub_name}")

pytorch_model_head.bin:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_adapter.bin:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

Pushed C_adapter_seq_bn_classification_C_20


In [29]:
import gc

print_gpu_utilization()

del classficiation_no_pretrain_model
del classification_trainer
gc.collect()

print_gpu_utilization()

GPU 12269 MB
GPU 12269 MB


# Classification from Pre-training

In [30]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
classficiation_from_pretrain_model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'heads.default.3.bias', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
load_name = f"BigTMiami/{pretrained_adapter_hub_name}"

# Load Pretrained adapter without head
loaded_adapter_name = classficiation_from_pretrain_model.load_adapter(load_name, with_head=False)

# Add head for classification modeling
classficiation_from_pretrain_model.add_classification_head(
    loaded_adapter_name,
    num_labels=2,
    id2label={ 0: "unhelpful", 1: "helpful"})

# Set the adapter to be used for training
classficiation_from_pretrain_model.train_adapter(loaded_adapter_name)

print(f"Classification Adapter Name: {loaded_adapter_name}")

classficiation_from_pretrain_model.active_adapters

(…)b/master/dist/v2/index/roberta-base.json:   0%|          | 0.00/540 [00:00<?, ?B/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

head_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

pytorch_adapter.bin:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

pytorch_model_head.bin:   0%|          | 0.00/157M [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

Classification Adapter Name: C_adapter_seq_bn_pretraining_P_15_adapter


Stack[C_adapter_seq_bn_pretraining_P_15_adapter]

In [32]:
from transformers import TrainingArguments

classification_from_pretrained_training_args = TrainingArguments(
    output_dir=classification_adapter_from_pretrained_hub_name,
    learning_rate=1e-4,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=250,
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

In [33]:
from adapters import AdapterTrainer

classification_from_pretrained_trainer = AdapterTrainer(
    model=classficiation_from_pretrain_model,
    args=classification_from_pretrained_training_args,
    train_dataset=classification_train_dataset,
    eval_dataset=classification_eval_dataset,
    data_collator=classification_data_collator,
    compute_metrics=compute_metrics,
)
print_gpu_utilization()

GPU 12269 MB


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [34]:
classification_from_pretrained_training_results = classification_from_pretrained_trainer.train()
print(classification_from_pretrained_training_results)
print_gpu_utilization()

Step,Training Loss
250,0.393100
500,0.359200
750,0.366600
1000,0.336700
1250,0.322700
1500,0.353300
1750,0.328100
2000,0.332700
2250,0.327900
2500,0.325900


TrainOutput(global_step=4323, training_loss=0.33131317625767154, metrics={'train_runtime': 1108.9493, 'train_samples_per_second': 62.356, 'train_steps_per_second': 3.898, 'total_flos': 1.7389360436066208e+16, 'train_loss': 0.33131317625767154, 'epoch': 3.0})
GPU 12289 MB


In [35]:
classification_from_pretrained_eval_results = classification_from_pretrained_trainer.evaluate()
classification_from_pretrained_eval_loss = classification_from_pretrained_eval_results["eval_loss"]
classification_from_pretrained_eval_f1 = classification_from_pretrained_eval_results["eval_f1_macro"]
print(f"Classification with Pretraining Eval F1:{100.0 * classification_from_pretrained_eval_f1:.2f}")
print(f"Classification with Pretraining Eval Loss: {classification_from_pretrained_eval_loss:.2f}")
print(classification_from_pretrained_eval_results)
print_gpu_utilization()

Classification with Pretraining Eval F1:61.36
Classification with Pretraining Eval Loss: 0.33
{'eval_loss': 0.3328656256198883, 'eval_accuracy': 0.865, 'eval_f1_macro': 0.6136194175304133, 'eval_runtime': 37.7721, 'eval_samples_per_second': 132.373, 'eval_steps_per_second': 8.287, 'epoch': 3.0}
GPU 12289 MB


In [ ]:
classficiation_from_pretrain_model.push_adapter_to_hub(
    classification_adapter_from_pretrained_hub_name,
    loaded_adapter_name,
    datasets_tag=classification_dataset_name
)
print(f"Pushed {classification_adapter_from_pretrained_hub_name}")

pytorch_model_head.bin:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_adapter.bin:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

In [ ]:
print_gpu_utilization()

In [ ]:
end_time = time()
total_time = end_time - start_time

In [ ]:
print("Summary")
print("----------------")
print(f"Adapter Type:{adapter_type}")
print(f"Pretrain Overall Train Percentage:{pretrain_percentage_final}")
print(f"Classification Train Percentage:{classification_train_percentage}")
print("")
print("Running Time")
print("----------------")
print(f"Total Time: {total_time:.0f} in seconds")
print(f"Total Time: {total_time / 60.0 :.0f} in minutes")
print("")
print("Pre-training Loss Change - should be high because adapter is completely random")
print("----------------")
print(f"Before pretraining loss: {before_pretrain_eval_results_loss:.4f}")
print(f"After pretraining loss: {after_pretrain_eval_results_loss:.4f}")
print("")
print("Classification Loss Change - should not be that different because both are starting from a blank classifier")
print("----------------")
print(f"Classification no Pretraining Eval Loss: {classification_eval_loss:.4f}")
print(f"Classification with Pretraining Eval Loss: {classification_from_pretrained_eval_loss:.4f}")
print("")
print("Classification F1 Change - hopefully we get a few points improvement here!")
print("----------------")
print(f"Classification no Pretraining Eval F1:{100.0 * classification_eval_f1:.2f}")
print(f"Classification with Pretraining Eval F1:{100.0 * classification_from_pretrained_eval_f1:.2f}")

print("")
pretraining_f1_improvement = (100.0 * (  classification_from_pretrained_eval_f1 - classification_eval_f1))
print(f"Pretraining F1 Improvement: {pretraining_f1_improvement:.2f}")

In [ ]:
print("Disconnecting Session")
from google.colab import runtime
runtime.unassign()